# SIAS Risk Performance Analysis

Code by Lili Gao

Last updated: Lili, July 31, 2025\
Last updated: Lili, Aug 12, 2025\
Last updated: Lili, Aug 27, 2025\
Last updated: Lili, Aug 29, 2025


# Project Overview

This project analyzes the **SIAS Global Equity Portfolio** relative to its benchmark (**iShares MSCI ACWI ETF**) to assess active positioning, risk exposures, key risk/return measures, and performance under scenarios.

## Key Steps & Outputs

1. **Portfolio & Benchmark Setup**

   * Load the latest **SIAS portfolio** (BNY Mellon) and **benchmark data** (iShares MSCI ACWI).
   * Produce summary tables of **assets, market values, prices, and base weights** (record the data **as-of date** for both).

2. **Current-Weight Diagnostics**

   * **Active Weights — Security level:** Differences vs. benchmark for each holding (over/under-weights).
   * **Active Exposures — Sector:** Sector tilts relative to the benchmark.
   * **Active Exposures — Region:** U.S. vs. ex-U.S. active weights (and other regions if relevant).

3. **Concentration Risk (HHI)**

   * Compute the **Herfindahl–Hirschman Index (HHI)** and **Effective Number of Positions**; highlight **Top-N concentration** and any single-name/sector concentration flags.

4. **Risk & Return Metrics**

   * **Volatility** (total return standard deviation)
   * **Beta (vs. ACWI)**
   * **Tracking Error** (active return volatility)
   * **Information Ratio** (active return / TE)
   * **Sharpe & Sortino Ratios**
   * **Maximum Drawdown**
   * **Value-at-Risk (VaR)** and **Expected Shortfall (ES)** at specified confidence levels

5. **Factor & Risk Model Insights**

   * Factor/risk model diagnostics as available (e.g., style, sector, country contributions).
   * **Stress-Test Scenarios:** Simulated P/L under adverse conditions (e.g., a –10% technology shock).

This workflow provides a comprehensive view of **active positioning**, **sector/region allocation differences**, **concentration**, and **scenario-driven performance**, supporting robust risk assessment and portfolio decision-making.

## Code

In [ ]:
# ======================================================================
# SIAS — Portfolio vs. ACWI (Benchmark) Interactive Views (Colab-ready)
# ======================================================================

from __future__ import annotations

# --- Setup & Imports ---------------------------------------------------
import sys, os, re, csv, warnings, glob
from pathlib import Path
from dataclasses import dataclass

import numpy as np
import pandas as pd
import yfinance as yf
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

TRADING_DAYS = 252
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_colwidth", 60)

# --- Colab mount (no-op on local) --------------------------------------
IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)

# --- Paths -------------------------------------------------------------
# Root folder that holds both the portfolio XLSX and the benchmark CSV
FOLDER = Path('/content/drive/MyDrive/Colab Notebooks/SIAS_Global Risk Monitoring/GE Risk Analysis_Shared Folder')

# Either put exact filenames, or leave "" and use a glob pattern below
PORT_FILENAME = '20250828 SIAS Asset Detail_29 Aug 2025.xls'   # e.g. exact file
BM_FILENAME   = 'ACWI_holdings_28 Aug 2025.csv'       # e.g. exact file
PORT_SHEET_NAME ="SIAS Asset Detail"

# If you prefer auto-pick the latest matching file, set these to "" and adjust patterns below:
PORT_GLOB = 'SIAS Asset Detail_*.xlsx'
BM_GLOB   = 'ACWI_holdings_*.csv'

def _latest(path_glob: str) -> Path | None:
    matches = sorted(FOLDER.glob(path_glob), key=lambda p: p.stat().st_mtime, reverse=True)
    return matches[0] if matches else None

# Build paths
if PORT_FILENAME:
    PORT_PATH = FOLDER / PORT_FILENAME
else:
    PORT_PATH = _latest(PORT_GLOB)

if BM_FILENAME:
    BM_PATH = FOLDER / BM_FILENAME
else:
    BM_PATH = _latest(BM_GLOB)

# --- Validation --------------------------------------------------------
def _require_exists(p: Path, label: str):
    if p is None:
        raise FileNotFoundError(f"{label}: no file found (check folder and glob).")
    if not p.exists():
        raise FileNotFoundError(f"{label} not found: {p}")
    if p.is_dir():
        raise IsADirectoryError(f"{label} is a directory, expected a file: {p}")

# Ensure base folder exists (especially on local runs)
if not FOLDER.exists():
    raise FileNotFoundError(f"Root folder not found: {FOLDER}\n- If in Colab, confirm Drive is mounted.\n- Otherwise set FOLDER correctly.")

_require_exists(PORT_PATH, "Portfolio file")
_require_exists(BM_PATH,   "Benchmark file")

print("Paths Loading:")
print(f" - Portfolio: {PORT_PATH.name}")
print(f" - Benchmark: {BM_PATH.name}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Paths Loading:
 - Portfolio: 20250828 SIAS Asset Detail_29 Aug 2025.xls
 - Benchmark: ACWI_holdings_28 Aug 2025.csv


In [ ]:
"""
SIAS USD-Equity vs ACWI Dashboard
----------------------------------------------------
Clean, step-by-step structure with clear section titles:

1) Assets details of SIAS portfolio and Benchmark
2) Active Weights — Security Level
3) Active Weights — Sector Level
4) Active Weights — Country Level
5) Concentration Metrics (HHI, Effective N, Top-N)
6) Risk Metrics (Vol, Sharpe, TE, IR, VaR/ES)
7) Sector Stress Test (Portfolio vs Benchmark)
8) Brinson–Fachler Attribution (Sector-level)

Notes
-----
* Designed for Jupyter/Colab. Requires: pandas, numpy, yfinance, ipywidgets.
* Set `PORT_PATH` and `BM_PATH` to your files. ACWI CSV usually has 9 metadata rows.
* USD equities are filtered from portfolio for a clean apples-to-apples slice.
"""

# ================================================
# helpers
# ================================================
def make_sort_ui(df, sort_columns, default_col, order_default='Descending'):
    """Simple sort + order controls above a table."""
    sort_dd  = widgets.Dropdown(options=list(sort_columns), value=default_col, description='Sort by:')
    order_tb = widgets.ToggleButtons(options=['Descending','Ascending'], value=order_default, description='Order:')
    out      = widgets.Output()
    def _update(*_):
        asc = (order_tb.value == 'Ascending')
        with out:
            clear_output(wait=True)
            display(df.sort_values(by=sort_dd.value, ascending=asc).reset_index(drop=True))
    _update()
    sort_dd.observe(_update, names='value')
    order_tb.observe(_update, names='value')
    return widgets.HBox([sort_dd, order_tb]), out, _update


def safe_upper_strip(s):
    return (str(s).strip().upper()) if pd.notna(s) else np.nan


def normalize_simple(s):
    if pd.isna(s): return ''
    t = str(s).lower()
    t = re.sub(r"[’'`]", "", t)
    t = re.sub(r"[^a-z0-9 ]+", " ", t)
    return re.sub(r"\s+", " ", t).strip()


def first_n_words(txt, n):
    t = normalize_simple(txt)
    return '' if not t else ' '.join(t.split()[:n])


def is_usd_like(x):
    if pd.isna(x): return False
    return str(x).upper().strip() in {"U.S. DOLLAR","US DOLLAR","USD","UNITED STATES DOLLAR"}


def fmt_int(x):
    try:
        return f"{int(round(float(x))):,}"
    except Exception:
        return "—"

# ================================================
# Data readers
# ================================================
@dataclass
class Benchmark:
    asof: pd.Timestamp
    holdings: pd.DataFrame  # columns: Ticker, Name, Sector, Price, Benchmark_Weight_%
    raw: pd.DataFrame       # original frame for country rollups


@dataclass
class Portfolio:
    asof: pd.Timestamp
    full: pd.DataFrame      # original portfolio frame
    usd_equity: pd.DataFrame  # filtered USD equities with computed weights & prices


def read_benchmark(bm_path: Path) -> Benchmark:
    """Read ACWI-like CSV: detects B2 as-of date and holdings from row 10."""
    # 1) as-of from B2
    bm_asof = pd.NaT
    try:
        with open(bm_path, 'r', encoding='utf-8', newline='') as f:
            rdr = list(csv.reader(f))
            if len(rdr) >= 2 and len(rdr[1]) >= 2:
                b2_text = rdr[1][1]
                bm_asof = pd.to_datetime(b2_text, errors='coerce')
                if pd.notna(bm_asof):
                    bm_asof = bm_asof.normalize()
    except Exception:
        pass

    # 2) holdings
    df_bm_raw = pd.read_csv(bm_path, skiprows=9)
    # Normalize names
    norm = {c: re.sub(r"\s+", " ", str(c).strip()).upper() for c in df_bm_raw.columns}
    df_bm_raw.columns = [norm[c] for c in df_bm_raw.columns]

    df_bm = df_bm_raw.rename(columns={
        'WEIGHT (%)': 'Benchmark_Weight_%',
        'PRICE': 'Price',
        'SECTOR': 'Sector',
        'NAME': 'Name',
        'TICKER': 'Ticker',
    })

    req = ['Ticker','Name','Sector','Price','Benchmark_Weight_%']
    missing = [c for c in req if c not in df_bm.columns]
    if missing:
        raise ValueError(f"Benchmark file missing columns: {missing}")

    df_bm = df_bm[req].copy()
    df_bm['Ticker'] = df_bm['Ticker'].astype(str).map(safe_upper_strip)
    df_bm['Price']  = pd.to_numeric(df_bm['Price'], errors='coerce')
    df_bm['Benchmark_Weight_%'] = pd.to_numeric(df_bm['Benchmark_Weight_%'], errors='coerce').fillna(0.0)

    # Example fix: Sanofi ticker harmonization
    df_bm.loc[df_bm['Name'].astype(str).str.contains('sanofi', case=False, na=False), 'Ticker'] = 'SNY'

    # unique by ticker
    df_bm = df_bm.drop_duplicates(subset=['Ticker'], keep='first')

    # Name-based keys for fallback matching later
    df_bm['Name_norm'] = df_bm['Name'].apply(normalize_simple)
    df_bm['key2'] = df_bm['Name_norm'].apply(lambda x: ' '.join(x.split()[:2]) if x else '')
    df_bm['key1'] = df_bm['Name_norm'].apply(lambda x: x.split()[0] if x else '')

    # rescale weights to ~100
    w_sum = df_bm['Benchmark_Weight_%'].sum()
    if 0 < w_sum < 99 or w_sum > 101:
        df_bm['Benchmark_Weight_%'] *= (100.0 / w_sum)

    return Benchmark(bm_asof, df_bm, df_bm_raw)


def read_portfolio_usd_equities(port_path: Path, sheet_name: str) -> Portfolio:
    """Read SIAS Excel, pick As Of Date from column 'As Of Date' row 2, filter USD equities."""
    # Full sheet (we'll not load everything yet to avoid memory; first read head to get as-of)
    port_asof = pd.NaT
    try:
        df_top = pd.read_excel(port_path, sheet_name=sheet_name, nrows=2)
        for col in df_top.columns:
            if str(col).strip().lower() == 'as of date':
                val = df_top[col].iloc[0]
                port_asof = pd.to_datetime(val, errors='coerce')
                if pd.notna(port_asof):
                    port_asof = port_asof.normalize()
                break
    except Exception:
        pass

    # Load full
    df_port = pd.read_excel(port_path, sheet_name=sheet_name)

    # Normalize name column
    if 'Security Description 1' in df_port.columns and 'Name' not in df_port.columns:
        df_port = df_port.rename(columns={'Security Description 1': 'Name'})

    req = ['Ticker','Name','Shares/Par','Country Name','Local Currency Name','Asset Type','Base Market Value']
    missing = [c for c in req if c not in df_port.columns]
    if missing:
        raise ValueError(f"Portfolio file missing columns: {missing}")

    mask_usd_equity = (
        df_port['Local Currency Name'].apply(is_usd_like) &
        df_port['Asset Type'].astype(str).str.upper().eq('EQUITY')
    )
    df_usd = df_port.loc[mask_usd_equity, req].copy()

    # Cleaning
    df_usd['Ticker'] = df_usd['Ticker'].astype(str).map(safe_upper_strip)
    df_usd['Name']   = df_usd['Name'].astype(str)
    df_usd['Shares/Par']        = pd.to_numeric(df_usd['Shares/Par'], errors='coerce').fillna(0.0)
    df_usd['Base Market Value'] = pd.to_numeric(df_usd['Base Market Value'], errors='coerce')
    df_usd['Market_Value_USD']  = df_usd['Base Market Value']

    # Weights
    total_mv = pd.to_numeric(df_usd['Market_Value_USD'], errors='coerce').sum()
    df_usd['Portfolio_Weight_%'] = np.where(total_mv > 0, df_usd['Market_Value_USD'] / total_mv * 100, 0.0)

    return Portfolio(port_asof, df_port, df_usd)


# ================================================
# Prices & returns around anchor date
# ================================================
def attach_anchor_prices(df_usd: pd.DataFrame, anchor_dt: pd.Timestamp) -> pd.DataFrame:
    tickers = sorted(df_usd["Ticker"].dropna().unique().tolist())
    if not tickers:
        df_usd = df_usd.copy()
        df_usd[['Price_USD','LogRet_1D','LogRet_1W','LogRet_1M']] = np.nan
        return df_usd

    start = (anchor_dt - pd.Timedelta(days=60)).date()
    end   = (anchor_dt + pd.Timedelta(days=1)).date()  # yfinance end is exclusive

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        data = yf.download(tickers=tickers, start=start, end=end, interval="1d",
                           auto_adjust=False, threads=True, progress=False)

    # Normalize to Close frame
    if isinstance(data.columns, pd.MultiIndex):
        if "Close" not in data.columns.get_level_values(0):
            raise RuntimeError("No 'Close' in downloaded data.")
        close = data["Close"].copy()
    else:
        if "Close" not in data.columns:
            raise RuntimeError("No 'Close' in downloaded data.")
        close = data[["Close"]].rename(columns={"Close": tickers[0]})

    close = close.dropna(how="all")

    # pick last trading row <= anchor_dt
    anchor_row = close.loc[:pd.Timestamp(anchor_dt).tz_localize(None)].tail(1)
    if anchor_row.empty:
        anchor_row = close.tail(1)
    last = anchor_row.squeeze()

    def lag_logret(df, anchor_i, lag_rows):
        pos = df.index.get_loc(anchor_i)
        if isinstance(pos, slice):
            return pd.Series(np.nan, index=df.columns)
        ref_pos = pos - lag_rows
        if ref_pos >= 0:
            ref = df.iloc[ref_pos]
            return np.log(last / ref)
        return pd.Series(np.nan, index=df.columns)

    out = pd.DataFrame({"Ticker": last.index})
    out["Price_USD"] = pd.to_numeric(last.values, errors="coerce")
    anchor_idx = anchor_row.index[0]
    out["LogRet_1D"] = lag_logret(close, anchor_idx, 1).values
    out["LogRet_1W"] = lag_logret(close, anchor_idx, 5).values
    out["LogRet_1M"] = lag_logret(close, anchor_idx, 21).values

    return df_usd.merge(out, on="Ticker", how="left")


# ================================================
# Combine & matching helpers
# ================================================
def build_name_match_lookups(df_bm: pd.DataFrame):
    look2 = (df_bm[df_bm['key2']!='']
             .drop_duplicates('key2', keep='first')
             .set_index('key2')[['Sector','Benchmark_Weight_%']].to_dict('index'))
    look1 = (df_bm[df_bm['key1']!='']
             .drop_duplicates('key1', keep='first')
             .set_index('key1')[['Sector','Benchmark_Weight_%']].to_dict('index'))
    return look2, look1


def combine_and_match(df_usd: pd.DataFrame, df_bm: pd.DataFrame):
    look2, look1 = build_name_match_lookups(df_bm)

    df_combined = (
        df_usd[['Ticker','Name','Shares/Par','Country Name','Price_USD','Market_Value_USD','Portfolio_Weight_%']]
        .merge(df_bm[['Ticker','Sector','Benchmark_Weight_%']], on='Ticker', how='left', suffixes=('','_bm'))
    )
    df_combined['Matched_By'] = np.where(df_combined['Benchmark_Weight_%'].notna(), 'Ticker', 'Unmatched')

    # Step 1: first two words of name
    df_combined['key2'] = df_combined['Name'].apply(lambda x: first_n_words(x, 2))
    hits2 = df_combined['key2'].map(look2)
    have2 = hits2.notna()
    if have2.any():
        vals2 = hits2[have2].apply(pd.Series)
        idx2  = hits2.index[have2]
        df_combined.loc[idx2, ['Sector','Benchmark_Weight_%']] = vals2[['Sector','Benchmark_Weight_%']].to_numpy()
        df_combined.loc[idx2, 'Matched_By'] = 'Name_first2'

    # Step 2: first word for remaining
    mask_missing = df_combined['Benchmark_Weight_%'].isna()
    if mask_missing.any():
        df_combined.loc[:, 'key1'] = df_combined['Name'].apply(lambda x: first_n_words(x, 1))
        hits1 = df_combined.loc[mask_missing, 'key1'].map(look1)
        have1 = hits1.notna()
        if have1.any():
            vals1 = hits1[have1].apply(pd.Series)
            idx1  = hits1.index[have1]
            df_combined.loc[idx1, ['Sector','Benchmark_Weight_%']] = vals1[['Sector','Benchmark_Weight_%']].to_numpy()
            df_combined.loc[idx1, 'Matched_By'] = 'Name_first1'

    # Finalize
    df_combined['Sector'] = df_combined['Sector'].fillna('ETF holdings')
    df_combined['Benchmark_Weight_%'] = df_combined['Benchmark_Weight_%'].fillna(0.0)
    df_combined['Active_Weight_%'] = df_combined['Portfolio_Weight_%'] - df_combined['Benchmark_Weight_%']
    return df_combined


# ================================================
# 1) Assets details of SIAS portfolio and Benchmark
# ================================================
def section_1_assets(port: Portfolio, bm: Benchmark):
    sias_mv   = pd.to_numeric(port.usd_equity["Market_Value_USD"], errors="coerce").sum()
    bm_w_sum  = pd.to_numeric(bm.holdings["Benchmark_Weight_%"], errors="coerce").sum()
    count_sias = len(port.usd_equity)
    count_bm   = len(bm.holdings)

    header_html = f"""
    <div style="padding:10px 12px;border:1px solid #ddd;border-radius:8px;font-family:Inter,system-ui,Segoe UI,Roboto,Arial,sans-serif;">
      <div style="font-size:14px;margin-bottom:6px;">
        <b>Files:</b> Portfolio: {PORT_PATH.name} &nbsp;|&nbsp; Benchmark: {BM_PATH.name}
      </div>
      <div style="font-size:14px;margin-bottom:6px;">
        <b>As-of dates:</b>
        Portfolio: {'' if pd.isna(port.asof) else port.asof.date()} &nbsp;|&nbsp;
        Benchmark: {'' if pd.isna(bm.asof) else bm.asof.date()}
      </div>
      <div style="font-size:14px;margin-bottom:6px;">
        <b>Totals:</b> SIAS MV = {fmt_int(sias_mv)} &nbsp;|&nbsp; ACWI weight sum ≈ {bm_w_sum:0.2f}%
      </div>
      <div style="font-size:14px;">
        <b>Asset Counts:</b> SIAS (USD equities): {count_sias} &nbsp;|&nbsp; ACWI holdings: {count_bm}
      </div>
    </div>
    """
    display(HTML("<h2>1. Assets details of SIAS portfolio and Benchmark</h2>"))
    display(HTML(header_html))

    display(HTML("<h3>Benchmark (ACWI) — Holdings Snapshot</h3>"))
    bm_view_cols = ['Ticker','Name','Sector','Price','Benchmark_Weight_%']
    bm_controls, bm_out, _ = make_sort_ui(bm.holdings[bm_view_cols].copy(),
                                          sort_columns=['Price','Benchmark_Weight_%'],
                                          default_col='Benchmark_Weight_%')
    display(bm_controls, bm_out)


# ================================================
# 2) Active Weights — Security Level
# ================================================
def section_2_active_security(df_usd: pd.DataFrame, df_combined: pd.DataFrame):


    display(HTML("<h3>SIAS Portfolio (USD Equities) — Weights & Returns</h3>"))
    metrics = ['Price_USD','LogRet_1D','LogRet_1W','LogRet_1M','Market_Value_USD','Portfolio_Weight_%']
    port_cols = ['Ticker','Name','Shares/Par','Country Name'] + metrics
    port_controls, port_out, _ = make_sort_ui(df_usd[port_cols].copy(),
                                              sort_columns=metrics,
                                              default_col='Market_Value_USD')
    display(port_controls, port_out)
    display(HTML("<h2>2. Active Weights — Security Level</h2>"))
    # display(HTML("<h3>Active Weights — By Security</h3>"))
    active_cols = ['Ticker','Name','Sector','Price_USD','Market_Value_USD','Portfolio_Weight_%','Benchmark_Weight_%','Active_Weight_%','Matched_By']
    act_controls, act_out, _ = make_sort_ui(df_combined[active_cols].copy(),
                                            sort_columns=['Portfolio_Weight_%','Benchmark_Weight_%','Active_Weight_%','Price_USD','Market_Value_USD'],
                                            default_col='Active_Weight_%')
    display(act_controls, act_out)


# ================================================
# 3) Active Weights — Sector Level
# ================================================
def section_3_active_sector(df_combined: pd.DataFrame, bm: Benchmark):
    display(HTML("<h2>3. Active Weights — Sector Level</h2>"))
    port_sector = df_combined.groupby('Sector', dropna=False)['Portfolio_Weight_%'].sum().reset_index()
    bm_sector   = (bm.holdings.assign(Sector=bm.holdings['Sector'].fillna('ETF holdings'))
                       .groupby('Sector', dropna=False)['Benchmark_Weight_%'].sum().reset_index())
    sector_tbl  = port_sector.merge(bm_sector, on='Sector', how='outer').fillna(0.0)
    sector_tbl['Active_Sector_%'] = sector_tbl['Portfolio_Weight_%'] - sector_tbl['Benchmark_Weight_%']

    sec_cols = ['Sector','Portfolio_Weight_%','Benchmark_Weight_%','Active_Sector_%']
    sec_controls, sec_out, _ = make_sort_ui(sector_tbl[sec_cols].copy(),
                                            sort_columns=['Portfolio_Weight_%','Benchmark_Weight_%','Active_Sector_%','Sector'],
                                            default_col='Active_Sector_%')
    display(sec_controls, sec_out)


# ================================================
# 4) Active Weights — Country Level
# ================================================
def section_4_active_country(df_combined: pd.DataFrame, bm: Benchmark):
    display(HTML("<h2>4. Active Weights — Country Level</h2>"))

    # Portfolio countries
    port_country = (
        df_combined.groupby('Country Name', dropna=False)['Portfolio_Weight_%']
                   .sum().reset_index()
                   .rename(columns={'Country Name':'Country'})
    )
    # Remove NA or '-' countries
    port_country = port_country[port_country['Country'].notna() & (port_country['Country'].astype(str).str.strip().str.upper() != 'NA') & (port_country['Country'].astype(str).str.strip() != '-')]

    # Benchmark countries from raw
    bm_country = (
        bm.raw.groupby('LOCATION', dropna=False)['WEIGHT (%)']
              .sum().reset_index()
              .rename(columns={'LOCATION':'Country','WEIGHT (%)':'Benchmark_Weight_%'})
    )
    bm_country = bm_country[bm_country['Country'].notna() & (bm_country['Country'].astype(str).str.strip().str.upper() != 'NA') & (bm_country['Country'].astype(str).str.strip() != '-')]

    # Merge (case-insensitive)
    port_country['Country_lower'] = port_country['Country'].astype(str).str.strip().str.lower()
    bm_country['Country_lower']   = bm_country['Country'].astype(str).str.strip().str.lower()

    country_tbl = (
        port_country.merge(bm_country, on='Country_lower', how='outer', suffixes=('_port','_bm'))
    )
    country_tbl['Country'] = country_tbl['Country_port'].combine_first(country_tbl['Country_bm'])
    country_tbl = country_tbl[['Country','Portfolio_Weight_%','Benchmark_Weight_%']].fillna(0.0)

    # Remove rows with Country = NA or '-' again (safety)
    country_tbl = country_tbl[country_tbl['Country'].notna() & (country_tbl['Country'].astype(str).str.strip().str.upper() != 'NA') & (country_tbl['Country'].astype(str).str.strip() != '-')]

    country_tbl['Active_Country_%'] = country_tbl['Portfolio_Weight_%'] - country_tbl['Benchmark_Weight_%']
    country_tbl = country_tbl.sort_values('Active_Country_%', ascending=False)

    # Interactive table
    display(HTML("<h3>Country Allocation — Interactive Table</h3>"))
    country_controls, country_out, _ = make_sort_ui(
        country_tbl[['Country','Portfolio_Weight_%','Benchmark_Weight_%','Active_Country_%']].copy(),
        sort_columns=['Portfolio_Weight_%','Benchmark_Weight_%','Active_Country_%','Country'],
        default_col='Active_Country_%'
    )
    display(country_controls, country_out)

    # US vs Non-US summary
    def us_vs_non_us(df):
        df = df.copy()
        df['is_us'] = df['Country'].astype(str).str.strip().str.lower() == 'united states'
        summary = pd.DataFrame({
            'Group': ['United States', 'Non-United States'],
            'Portfolio_Weight_%': [df.loc[df['is_us'],  'Portfolio_Weight_%'].sum(), df.loc[~df['is_us'], 'Portfolio_Weight_%'].sum()],
            'Benchmark_Weight_%': [df.loc[df['is_us'],  'Benchmark_Weight_%'].sum(), df.loc[~df['is_us'], 'Benchmark_Weight_%'].sum()],
        })
        summary['Active_Country_%'] = summary['Portfolio_Weight_%'] - summary['Benchmark_Weight_%']
        return summary

    display(HTML("<h3>United States vs Non-United States — Summary</h3>"))
    display(us_vs_non_us(country_tbl))


# ================================================
# 5) Concentration Metrics
# ================================================
def section_5_concentration(df_combined: pd.DataFrame):
    display(HTML("<h2>5. Concentration Metrics</h2>"))

    w_port_frac = (df_combined['Portfolio_Weight_%']  / 100.0).fillna(0.0)
    w_bm_frac   = (df_combined['Benchmark_Weight_%'] / 100.0).fillna(0.0)

    def hhi(weights: pd.Series) -> float:
        w = pd.to_numeric(weights, errors='coerce').fillna(0.0)
        return float(np.square(w).sum())

    def effective_n(weights: pd.Series) -> float:
        h = hhi(weights)
        return np.inf if h == 0 else 1.0 / h

    def top_n_weight(weights: pd.Series, n: int = 10) -> float:
        w = pd.to_numeric(weights, errors='coerce').fillna(0.0)
        n = max(1, int(n))
        return float(w.sort_values(ascending=False).head(n).sum())

    N_slider = widgets.IntSlider(value=10, min=1, max=30, step=1, description='Top-N:')
    out = widgets.Output()

    def _show(n):
        with out:
            clear_output(wait=True)
            metrics = pd.DataFrame({
                'Portfolio' : [hhi(w_port_frac),     effective_n(w_port_frac),     top_n_weight(w_port_frac, n)*100],
                'Benchmark' : [hhi(w_bm_frac),       effective_n(w_bm_frac),       top_n_weight(w_bm_frac, n)*100],
            }, index=['HHI','Effective #','Top-N %'])
            display(metrics.round(4))

    N_slider.observe(lambda ch: _show(ch['new']), names='value')
    _show(N_slider.value)
    display(N_slider, out)


# ================================================
# 6) Risk Metrics
# ================================================
def section_6_risk(df_combined: pd.DataFrame):
    display(HTML("<h2>6. Risk Metrics</h2>"))

    tickers_risk = list(pd.Index(df_combined['Ticker']).dropna().astype(str).unique()) + ['ACWI']
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        px = yf.download(tickers_risk, period='5y', interval='1d', auto_adjust=True, progress=False)['Close']

    if isinstance(px.columns, pd.MultiIndex):
        px = px.droplevel(0, axis=1)

    px = px.dropna(how='all').ffill()
    rets = px.pct_change().dropna(how='all')
    if 'ACWI' not in rets.columns:
        raise ValueError("ACWI benchmark series missing from downloaded data.")

    w_port = (df_combined.set_index('Ticker')['Portfolio_Weight_%'] / 100.0)
    w_port = w_port.groupby(level=0).sum().reindex(rets.columns, fill_value=0.0)
    if w_port.sum() > 0:
        w_port = w_port / w_port.sum()

    port_ret = rets.dot(w_port).dropna()
    bm_ret   = rets['ACWI'].dropna()
    port_ret, bm_ret = port_ret.align(bm_ret, join='inner')

    def volatility(r):
        return float(r.std(ddof=1) * np.sqrt(TRADING_DAYS))
    def beta(r, m):
        rr, mm = r.align(m, join='inner')
        v = float(mm.var(ddof=1))
        if v == 0: return np.nan
        c = float(np.cov(rr, mm, ddof=1)[0,1])
        return c / v
    def tracking_error(r, m):
        d = r.align(m, join='inner')
        diff = d[0] - d[1]
        return float(diff.std(ddof=1) * np.sqrt(TRADING_DAYS))
    def info_ratio(r, m):
        d = r.align(m, join='inner')
        diff = d[0] - d[1]
        te = float(diff.std(ddof=1))
        if te == 0: return np.nan
        return float((diff.mean() / te) * np.sqrt(TRADING_DAYS))
    def sharpe(r, rf_annual=0.0):
        rf_daily = rf_annual / TRADING_DAYS
        ex = r - rf_daily
        vol = float(ex.std(ddof=1))
        if vol == 0: return np.nan
        return float((ex.mean() / vol) * np.sqrt(TRADING_DAYS))
    def sortino(r, rf_annual=0.0):
        rf_daily = rf_annual / TRADING_DAYS
        ex = r - rf_daily
        down = ex[ex < 0]
        dstd = float(down.std(ddof=1))
        if dstd == 0: return np.nan
        return float((ex.mean() / dstd) * np.sqrt(TRADING_DAYS))
    def max_drawdown(r):
        cum = (1 + r).cumprod()
        peak = cum.cummax()
        dd = (cum - peak) / peak
        return float(dd.min())
    def var_es(r, p=0.95):
        r = r.dropna()
        if r.empty:
            return np.nan, np.nan
        var = float(np.percentile(r, (1 - p) * 100))
        tail = r[r <= var]
        es = float(tail.mean()) if len(tail) else np.nan
        return var, es

    windows = {'1m':21, '3m':63, '6m':126, '12m':252}
    win_dd = widgets.Dropdown(options=list(windows.keys()), value='3m', description='Window:')
    rf_slider   = widgets.FloatSlider(value=0.00, min=0.00, max=0.05, step=0.001,
                                      description='RF (ann):', readout_format='.3f')
    out = widgets.Output()

    def build_metrics(label, rf_ann):
        N = windows[label]
        pr = port_ret.tail(N)
        br = bm_ret.tail(N)
        VaR_p, ES_p = var_es(pr)
        VaR_b, ES_b = var_es(br)
        data = {
            'Return %'         : [ (1+pr).prod()-1, (1+br).prod()-1 ],
            'Volatility %'     : [ volatility(pr), volatility(br) ],
            'Sharpe'           : [ sharpe(pr, rf_ann),  sharpe(br, rf_ann) ],
            'Sortino'          : [ sortino(pr, rf_ann), sortino(br, rf_ann) ],
            'Max Drawdown %'   : [ max_drawdown(pr),    max_drawdown(br) ],
            'VaR(95) %'        : [ VaR_p, VaR_b ],
            'ES(95) %'         : [ ES_p,  ES_b ],
            'Beta vs ACWI'     : [ beta(pr, br), np.nan ],
            'Tracking Error %' : [ tracking_error(pr, br), np.nan ],
            'Information Ratio': [ info_ratio(pr, br),    np.nan ],
        }
        return pd.DataFrame(data, index=['Portfolio','Benchmark']).astype(float).round(3) * 100

    def _update(_=None):
        with out:
            clear_output(wait=True)
            display(build_metrics(win_dd.value, rf_slider.value))

    win_dd.observe(_update, names='value')
    rf_slider.observe(_update, names='value')
    _update()

    display(HTML("<h3>Risk Metrics — Interactive</h3>"))
    display(widgets.HBox([win_dd, rf_slider]), out)


# new
def section_6_risk(df_combined):
    # ── Imports local to function to avoid missing symbols ────────────────────
    import warnings, numpy as np, pandas as pd
    import yfinance as yf
    import ipywidgets as widgets
    from IPython.display import display, HTML, clear_output

    TRADING_DAYS = 252  # define locally

    display(HTML("<h2>6. Risk Metrics</h2>"))

    # ---------- 1) Download prices ----------
    # Build ticker list and ensure ACWI exists exactly once
    tickers = pd.Index(df_combined.get('Ticker', [])).dropna().astype(str).str.upper().unique().tolist()
    if 'ACWI' not in tickers:
        tickers.append('ACWI')
    else:
        # ensure no duplicates (yfinance handles, but we keep it clean)
        tickers = list(pd.unique(tickers))

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        px = yf.download(tickers, period='5y', interval='1d',
                         auto_adjust=True, progress=False)['Close']

    # yfinance can return:
    # - DataFrame with tickers as columns
    # - Series if a single ticker
    # - MultiIndex columns in some versions (e.g., ('Close', 'AAPL')) – we handle both
    if isinstance(px, pd.Series):
        px = px.to_frame(name=tickers[0])
    if isinstance(px.columns, pd.MultiIndex):
        # keep only the ticker level if ('Close', TICKER)
        try:
            px = px.droplevel(0, axis=1)
        except Exception:
            pass

    # Clean up
    px = px.dropna(how='all').ffill().dropna(axis=1, how='all')

    if 'ACWI' not in px.columns:
        raise ValueError("ACWI benchmark series missing from downloaded data.")

    rets = px.pct_change().dropna(how='all')
    if rets.empty:
        raise ValueError("No return series available after cleaning.")

    # ---------- 2) Portfolio weights aligned to price columns ----------
    w_port = (
        df_combined
        .assign(Ticker=lambda d: d['Ticker'].astype(str).str.upper())
        .set_index('Ticker')['Portfolio_Weight_%']
        .div(100.0)
        .groupby(level=0).sum()
        .reindex(rets.columns, fill_value=0.0)
    )
    eff_weight_sum = w_port.sum()
    if eff_weight_sum <= 0:
        raise ValueError("No overlap between portfolio tickers and downloaded price series.")
    w_port = w_port / eff_weight_sum

    # ---------- 3) Series for portfolio & benchmark ----------
    port_ret = rets.dot(w_port).dropna()
    bm_ret   = rets['ACWI'].dropna()
    port_ret, bm_ret = port_ret.align(bm_ret, join='inner')

    # ---------- 4) Metric functions ----------
    def volatility(r):
        return float(r.std(ddof=1) * np.sqrt(TRADING_DAYS))       # annualized

    def beta(r, m):
        rr, mm = r.align(m, join='inner')
        v = float(mm.var(ddof=1))
        if v == 0: return np.nan
        c = float(np.cov(rr, mm, ddof=1)[0, 1])
        return c / v

    def tracking_error(r, m):
        rr, mm = r.align(m, join='inner')
        diff = rr - mm
        return float(diff.std(ddof=1) * np.sqrt(TRADING_DAYS))    # annualized

    def info_ratio(r, m):
        rr, mm = r.align(m, join='inner')
        diff = rr - mm
        te_daily = float(diff.std(ddof=1))
        if te_daily == 0: return np.nan
        return float((diff.mean() / te_daily) * np.sqrt(TRADING_DAYS))  # annualized IR

    def sharpe(r, rf_annual=0.0):
        rf_daily = rf_annual / TRADING_DAYS
        ex = r - rf_daily
        vol_d = float(ex.std(ddof=1))
        if vol_d == 0: return np.nan
        return float((ex.mean() / vol_d) * np.sqrt(TRADING_DAYS)) # annualized

    def sortino(r, rf_annual=0.0):
        rf_daily = rf_annual / TRADING_DAYS
        ex = r - rf_daily
        down = ex[ex < 0]
        dstd = float(down.std(ddof=1))
        if dstd == 0: return np.nan
        return float((ex.mean() / dstd) * np.sqrt(TRADING_DAYS))  # annualized

    def max_drawdown(r):
        cum = (1 + r).cumprod()
        peak = cum.cummax()
        dd = (cum - peak) / peak
        return float(dd.min())                                    # period MDD (negative)

    def var_es(r, p=0.95):
        r = r.dropna()
        if r.empty:
            return np.nan, np.nan
        # Historical daily VaR/ES at confidence p
        var = float(np.quantile(r, 1 - p))
        tail = r[r <= var]
        es = float(tail.mean()) if len(tail) else np.nan
        return var, es

    # ---------- 5) Interactive build ----------
    windows = {'1m': 21, '3m': 63, '6m': 126, '12m': 252}
    win_dd = widgets.Dropdown(options=list(windows.keys()), value='3m', description='Window:')
    rf_slider = widgets.FloatSlider(value=0.00, min=0.00, max=0.05, step=0.001,
                                    description='RF (ann):', readout_format='.3f')
    out = widgets.Output()

    # Columns we will format as percentages
    percent_cols = ['Return (period) %', 'Volatility %', 'Max Drawdown %',
                    'VaR (95%, daily) %', 'ES (95%, daily) %', 'Tracking Error %']

    def build_metrics(label, rf_ann):
        N = windows[label]
        pr = port_ret.tail(N)
        br = bm_ret.tail(N)

        # Daily historical VaR/ES based on the selected window
        VaR_p, ES_p = var_es(pr, p=0.95)
        VaR_b, ES_b = var_es(br, p=0.95)

        df = pd.DataFrame({
            'Return (period) %' : [(1 + pr).prod() - 1,      (1 + br).prod() - 1],  # cumulative over window
            'Volatility %'      : [volatility(pr),           volatility(br)],       # annualized
            'Sharpe'            : [sharpe(pr, rf_ann),       sharpe(br, rf_ann)],
            'Sortino'           : [sortino(pr, rf_ann),      sortino(br, rf_ann)],
            'Max Drawdown %'    : [max_drawdown(pr),         max_drawdown(br)],
            'VaR (95%, daily) %': [VaR_p,                    VaR_b],
            'ES (95%, daily) %' : [ES_p,                     ES_b],
            'Beta vs ACWI'      : [beta(pr, br),             np.nan],
            'Tracking Error %'  : [tracking_error(pr, br),   np.nan],
            'Information Ratio' : [info_ratio(pr, br),       np.nan],
        }, index=['Portfolio', 'Benchmark'])

        # Nice formatting: only percent_cols as percents; ratios as numbers
        fmt = {c: '{:.2%}' for c in percent_cols}
        fmt.update({'Sharpe': '{:.3f}', 'Sortino': '{:.3f}', 'Beta vs ACWI': '{:.3f}', 'Information Ratio': '{:.3f}'})
        return df.style.format(fmt).set_table_styles(
            [{'selector': 'th.col_heading', 'props': [('text-align', 'center')]},
             {'selector': 'th.row_heading', 'props': [('text-align', 'left')]}]
        )

    def _update(_=None):
        with out:
            clear_output(wait=True)
            display(build_metrics(win_dd.value, rf_slider.value))

    win_dd.observe(_update, names='value')
    rf_slider.observe(_update, names='value')

    display(HTML("<h3>Risk Metrics — Interactive</h3>"))
    _update()  # initial render
    display(widgets.HBox([win_dd, rf_slider]), out)

# ================================================
# 7) Sector Stress Test (Portfolio vs Benchmark)
# ================================================
def section_7_stress(df_combined: pd.DataFrame, bm: Benchmark):
    display(HTML("<h2>7. Sector Stress Test (Portfolio vs Benchmark)</h2>"))

    # helpers
    def _signed_span(val: float) -> str:
        if pd.isna(val):
            return f"<span>{val}</span>"
        color = "red" if val > 0 else ("green" if val < 0 else "#555")
        return f"<span style='color:{color};font-weight:600'>{val:+.2f}%</span>"

    def _gain_red_loss_green(val):
        if pd.isna(val): return ""
        if val > 0:   return "background-color: #ffd6d6"
        if val < 0:   return "background-color: #d6ffe2"
        return "background-color: #f2f2f2"

    sector_map = df_combined.groupby('Ticker')['Sector'].first()
    w_for_stress = (df_combined.set_index('Ticker')['Portfolio_Weight_%'] / 100.0).groupby(level=0).sum()
    bm_sector_w = (bm.holdings.assign(Sector=bm.holdings['Sector'].fillna('ETF holdings'))
                       .groupby('Sector', dropna=False)['Benchmark_Weight_%']
                       .sum().rename('Bench_Weight_%'))
    bm_sector_w_frac = (bm_sector_w / 100.0)

    def sector_stress_test_vs_bench(sector_shocks, port_weights_by_ticker, sector_map, bench_sector_weights_frac):
        aligned_weights = port_weights_by_ticker.reindex(sector_map.index, fill_value=0.0)
        tick_sector     = sector_map.reindex(aligned_weights.index)
        shock_vec       = tick_sector.map(sector_shocks).fillna(0.0)
        pl_ticker       = aligned_weights * shock_vec
        total_port_pct  = float(pl_ticker.sum() * 100)

        port_by_sec_pct = (pl_ticker * 100).groupby(tick_sector).sum()
        bench_shock_series = pd.Series(sector_shocks, dtype=float)
        bench_by_sec_pct   = (bench_sector_weights_frac * bench_shock_series).fillna(0.0) * 100.0
        total_bench_pct    = float(bench_by_sec_pct.sum())

        all_secs = sorted(set(port_by_sec_pct.index.astype(str)) | set(bench_by_sec_pct.index.astype(str)))
        port_by_sec_al  = port_by_sec_pct.reindex(all_secs, fill_value=0.0)
        bench_by_sec_al = bench_by_sec_pct.reindex(all_secs, fill_value=0.0)
        active_by_sec   = port_by_sec_al - bench_by_sec_al
        total_active_pct = float(total_port_pct - total_bench_pct)

        port_sec_w = (df_combined.groupby('Sector', dropna=False)['Portfolio_Weight_%'].sum().rename('Port_Sector_Weight_%'))
        bench_sec_w = bm_sector_w.rename('Bench_Sector_Weight_%')

        sector_df = (pd.DataFrame({
                        'Sector'        : all_secs,
                        'Port_pctpts'   : port_by_sec_al.values,
                        'Bench_pctpts'  : bench_by_sec_al.values,
                        'Active_pctpts' : active_by_sec.values,
                     })
                     .merge(port_sec_w.reset_index().rename(columns={'Sector':'Sector'}), on='Sector', how='left')
                     .merge(bench_sec_w.reset_index().rename(columns={'Sector':'Sector'}), on='Sector', how='left')
                     .fillna(0.0))
        sector_df = sector_df.sort_values('Active_pctpts', ascending=False)

        ticker_df = (pl_ticker * 100).sort_values(ascending=False).reset_index()
        ticker_df.columns = ['Ticker', 'Pct-pts']
        tkr_names = df_combined.set_index('Ticker')['Name'].groupby(level=0).first()
        ticker_df = ticker_df.merge(tkr_names.rename('Name').reset_index(), on='Ticker', how='left')
        return total_port_pct, total_bench_pct, total_active_pct, sector_df, ticker_df

    unique_sectors = sorted(sector_map.dropna().unique().tolist())
    slider_dict = {sec: widgets.FloatSlider(value=0.0, min=-0.20, max=0.20, step=0.01,
                                            description=sec[:24], readout_format='.0%',
                                            layout=widgets.Layout(width='380px'))
                   for sec in unique_sectors}
    ui_sliders = widgets.VBox([widgets.HTML("<h4 style='margin:0 0 6px 0;'>Sector shocks (daily return)</h4>")] + list(slider_dict.values()),
                              layout=widgets.Layout(border='1px solid #ddd', padding='8px', width='410px'))

    def _style_sector_table(df: pd.DataFrame):
        def _apply_colors(s):
            return [_gain_red_loss_green(v) if s.name in ('Port_pctpts','Bench_pctpts','Active_pctpts') else '' for v in s]
        sty = (df.style
               .format({'Port_pctpts':'{:+.2f}','Bench_pctpts':'{:+.2f}','Active_pctpts':'{:+.2f}',
                        'Port_Sector_Weight_%':'{:.2f}%','Bench_Sector_Weight_%':'{:.2f}%'} )
               .hide(axis='index')
               .apply(_apply_colors, subset=['Port_pctpts'])
               .apply(_apply_colors, subset=['Bench_pctpts'])
               .apply(_apply_colors, subset=['Active_pctpts']))
        return sty

    def _two_tables(left_df, right_df, left_title="Top + Tickers (Port)", right_title="Top – Tickers (Port)"):
        sty_pos = (left_df.style.format({'Pct-pts':'{:+.2f}'}).hide(axis='index')
                  .apply(lambda s: [_gain_red_loss_green(v) if s.name=='Pct-pts' else '' for v in s], axis=0))
        sty_neg = (right_df.style.format({'Pct-pts':'{:+.2f}'}).hide(axis='index')
                  .apply(lambda s: [_gain_red_loss_green(v) if s.name=='Pct-pts' else '' for v in s], axis=0))
        left_html  = sty_pos.set_caption(left_title).to_html()
        right_html = sty_neg.set_caption(right_title).to_html()
        return HTML(f"""
        <div style="display:flex; gap:16px;">
          <div style="flex:1; min-width:0;">{left_html}</div>
          <div style="flex:1; min-width:0;">{right_html}</div>
        </div>
        """)

    out_stress = widgets.Output()

    def _refresh(*_):
        shocks = {sec: sl.value for sec, sl in slider_dict.items()}
        tot_p, tot_b, tot_a, sector_df, ticker_df = sector_stress_test_vs_bench(
            shocks, w_for_stress, sector_map, bm_sector_w_frac
        )
        with out_stress:
            clear_output(wait=True)
            display(HTML(f"""
            <div style="border:1px solid #ddd; border-radius:8px; padding:10px 12px; margin-bottom:10px;">
              <div style="font-size:14px; margin-bottom:4px;"><b>Portfolio:</b> {_signed_span(tot_p)}</div>
              <div style="font-size:14px; margin-bottom:4px;"><b>Benchmark:</b> {_signed_span(tot_b)}</div>
              <div style="font-size:14px;"><b>Active (Port − Bench):</b> {_signed_span(tot_a)}</div>
            </div>
            """))
            display(HTML("<h4 style='margin:8px 0 4px 0;'>By Sector — Portfolio vs Benchmark (pct-points)</h4>"))
            display(_style_sector_table(sector_df[['Sector','Port_pctpts','Bench_pctpts','Active_pctpts','Port_Sector_Weight_%','Bench_Sector_Weight_%']]))
            top_pos = ticker_df.sort_values('Pct-pts', ascending=False).head(10)[['Ticker','Name','Pct-pts']]
            top_neg = ticker_df.sort_values('Pct-pts', ascending=True ).head(10)[['Ticker','Name','Pct-pts']]
            display(HTML("<h4 style='margin:12px 0 4px 0;'>Ticker Contributions — Portfolio (pct-points)</h4>"))
            display(_two_tables(top_pos, top_neg))

    for sl in slider_dict.values():
        sl.observe(_refresh, names='value')
    _refresh()

    display(HTML("<h3>Sector Shocks — Interactive</h3>"))
    display(widgets.HBox([ui_sliders, widgets.VBox([out_stress], layout=widgets.Layout(flex='1'))],
                         layout=widgets.Layout(align_items='flex-start', gap='16px')))


# ================================================
# Main driver
# ================================================

def main():
    # Read inputs
    bm = read_benchmark(BM_PATH)
    port = read_portfolio_usd_equities(PORT_PATH, PORT_SHEET_NAME)

    # Anchor date preference: portfolio -> benchmark -> today
    anchor = port.asof if pd.notna(port.asof) else (bm.asof if pd.notna(bm.asof) else pd.Timestamp.today().normalize())

    # Attach prices/short-window returns to portfolio slice
    port_usd = attach_anchor_prices(port.usd_equity, anchor)

    # Combine with benchmark + fuzzy name matching for sectors/weights
    df_combined = combine_and_match(port_usd, bm.holdings)

    # Sections
    section_1_assets(port, bm)
    section_2_active_security(port_usd, df_combined)
    section_3_active_sector(df_combined, bm)
    section_4_active_country(df_combined, bm)
    section_5_concentration(df_combined)
    section_6_risk(df_combined)
    section_7_stress(df_combined, bm)

# Uncomment to run all sections in one go inside a notebook:
main()

Output()

Output()

Output()

Output()

Output()

,Group,Portfolio_Weight_%,Benchmark_Weight_%,Active_Country_%
0,United States,68.809316,64.49,4.319316
1,Non-United States,31.190684,34.83,-3.639316


IntSlider(value=10, description='Top-N:', max=30, min=1)

Output()

Output()